In [ ]:
pip install torch -U

In [1]:
import torch
import esm
from model import SparseForTokenClassification
from typing import Tuple, List


### Extracting Embedding for Both Protein Sequences and for Genome
#####  Extracting protien for a single protein 


In [2]:
_, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
batch_converter = alphabet.get_batch_converter()


In [3]:
# Usign a short 10 amino acid sequence as a proof of concept.

data  = [('seq_1', 'MLKKLSVFLI')]

batch_labels, batch_strs, batch_tokens = batch_converter(data)

##### Recreate the model

In [4]:
version = '3C' # or '5B'
checkpoint = torch.load(f'./models/{version}/config_and_model.pth', map_location='cpu', weights_only=False)

In [5]:
device = "cpu"
config = checkpoint['config']
model_state_dict = checkpoint['model_state_dict']
sparse_model = SparseForTokenClassification(config=config)
sparse_model.load_state_dict(model_state_dict)
sparse_model = sparse_model.to(device)
sparse_model = sparse_model.eval()

In [6]:
with torch.no_grad():
    output = sparse_model(input_ids=batch_tokens, output_attentions = True)
             
             
                     

In [7]:
output.logits.size()

torch.Size([1, 12, 1280])

### We can extract the sequence embeddings using


In [ ]:
output.logits[0,1:-1:].size()

In [8]:
len(output.attentions)

16

In [9]:
[output.attentions[i].size() for i in range(len(output.attentions))]

[torch.Size([1, 20, 12, 12]),
 torch.Size([1, 20, 12, 12]),
 torch.Size([1, 20, 12, 12]),
 torch.Size([1, 20, 12, 12]),
 torch.Size([1, 20, 12, 12]),
 torch.Size([1, 20, 12, 12]),
 torch.Size([1, 20, 12, 12]),
 torch.Size([1, 20, 12, 12]),
 torch.Size([1, 20, 12, 12]),
 torch.Size([1, 20, 12, 12]),
 torch.Size([1, 20, 12, 12]),
 torch.Size([1, 20, 12, 12]),
 torch.Size([1, 20, 12, 12]),
 torch.Size([1, 20, 12, 12]),
 torch.Size([1, 20, 12, 12]),
 torch.Size([1, 20, 12, 12])]

# THIBAUT

Could you include the code here that combines all the attention heads and layers into a single 10x10 matrix? It should be in a single function. For now, we will provide equal weights to each matrix, but the function should be able to take a different weighting scheme for each layer. See scaffold below


In [ ]:

def compute_combined_attention(attentions: Tuple[torch.Tensor, ...], weight_scheme: List[float]) -> torch.Tensor:
    pass

### Computing the Complete Genome Annotation

In [31]:
data  = [('seq_1', 'MLKKLSVFLI'), ('seq_2', 'MLSVFLI'), ('seq_3', 'LKKLSVFLI'), ('seq_4', 'MLKKLSVIMMMKV')]

# concat all the sequences in data
prots_concatenated = "".join([x[1] for x in data])
prots_lengths = [len(x[1]) for x in data]
genome = [("some_genome", prots_concatenated)]
genome , prots_lengths

([('some_genome', 'MLKKLSVFLIMLSVFLILKKLSVFLIMLKKLSVIMMMKV')], [10, 7, 9, 13])

In [32]:
len(genome[0][1])

39

In [33]:
batch_labels, batch_strs, batch_tokens = batch_converter(genome)

In [34]:
output = sparse_model(input_ids=batch_tokens, 
                      output_attentions = True, 
                      proteins_sizes = torch.tensor(prots_lengths))


In [35]:
output.keys()

odict_keys(['logits', 'attentions'])

In [36]:
output['logits'].size()

torch.Size([1, 41, 1280])

In [37]:
len(output['attentions'])

16

In [38]:
output['attentions'][0].size()

torch.Size([1, 20, 41, 41])

In [42]:
sum(prots_lengths)

39

In [45]:
# THIBAUT: my understanding is that two_step_selection= True determines whether to return all pairwise scores  or  not.
# Adding here, causes an error.


In [44]:
output = sparse_model(input_ids=batch_tokens, 
                      output_attentions = True, 
                      proteins_sizes = torch.tensor(prots_lengths),  two_step_selection= True)


ZeroDivisionError: float division by zero